In [ ]:
from src.loader import NewsDataLoader
from sklearn.decomposition import LatentDirichletAllocation

data_directory = "../data"
loader = NewsDataLoader(data_directory)
merge_df = loader.load_data()

tfidf_vectorizer = TfidfVectorizer(max_features=1000)
tfidf_matrix = tfidf_vectorizer.fit_transform(merge_df['content'])

lda = LatentDirichletAllocation(n_components=10, random_state=42)
lda.fit(tfidf_matrix)

# Implement topic modeling using LDA
# Categorize titles/content into topic categories
# Analyze topics and trends
# Visualize results
